Statisitcal Learning and Data Mining 

Assignment

Name : Shubham Jagtap

Student ID : N19091023

In [1]:
import pandas as pd

df = pd.read_csv("movies.csv",encoding='ISO-8859-1')
df.shape

(6820, 15)

Budget value seeems to be missing from the dataset. We need to impute those values. Budget may vary from company to company, also with respect to the genre of the movie and also with respect to the year it was released. We need to verify the factors budget depends on and impute it accordingly.

In [2]:
df['budget'].value_counts()

0.0            2182
20000000.0      207
30000000.0      191
25000000.0      177
15000000.0      170
               ... 
29000001.0        1
128000000.0       1
24500000.0        1
300000.0          1
5800000.0         1
Name: budget, Length: 351, dtype: int64

2182 films don't have any budget value. We need to verify the relation of budget values with other variables like the year, company and genre.

In [3]:
df3 = df.loc[df['budget'] != 0]
df3.corr()

,budget,gross,runtime,score,votes,year
budget,1.000000,0.680033,0.313064,0.073579,0.451467,0.259835
gross,0.680033,1.000000,0.253273,0.229552,0.642904,0.167492
runtime,0.313064,0.253273,1.000000,0.417031,0.359817,0.047811
score,0.073579,0.229552,0.417031,1.000000,0.470648,0.061479
votes,0.451467,0.642904,0.359817,0.470648,1.000000,0.184096
year,0.259835,0.167492,0.047811,0.061479,0.184096,1.000000


There is less correlation between year and the budget of the film. However, there is some correlation between gross and budget. Even though gross has a high correlation with budget we cannot use it to impute budget values, as it is a response variable for Q.1 a.

In [4]:
company_grp = df3.groupby(['company'])
company_grp['budget'].agg('mean').sort_values()

company
Next Wave Films           6.000000e+03
ERBP                      7.000000e+03
Can I Watch               1.500000e+04
Solana Films              1.500000e+04
Professor Bright Films    2.000000e+04
                              ...     
H2F Entertainment         1.750000e+08
Halcyon Company, The      2.000000e+08
C-2 Pictures              2.000000e+08
Marvel Studios            2.035714e+08
B24                       2.450000e+08
Name: budget, Length: 1340, dtype: float64

There is high difference in budget for different companies. Next Wave Films and B24 have huge difference in budgets.

In [5]:
genre_grp = df3.groupby(['genre'])

In [6]:
genre_grp['budget'].agg('mean')

genre
Action       5.921049e+07
Adventure    5.024821e+07
Animation    7.781004e+07
Biography    2.546938e+07
Comedy       2.369318e+07
Crime        2.386503e+07
Drama        2.415350e+07
Family       1.992167e+07
Fantasy      1.739455e+07
Horror       1.439298e+07
Musical      1.000000e+07
Mystery      3.290484e+07
Romance      1.644000e+07
Sci-Fi       1.995455e+07
Thriller     1.325000e+07
Western      6.000000e+06
Name: budget, dtype: float64

Compared to company wise budget the genre wise budget has smaller range. Imputing values with respect to company seems more reasonable.

Let us try to impute the missing values in budget by substituting the missing value with the average budget of the company for different films.

In [7]:
cpy_g = df.groupby(['company'])

In [8]:
a = company_grp.get_group('Columbia Pictures Corporation').mean()

In [9]:
df[df['budget']==0.0]['company']

25                   TriStar Pictures
26               Neue Constantin Film
27                   TriStar Pictures
32                            Gaumont
35      Columbia Pictures Corporation
                    ...              
6813          Anna Biller Productions
6815         Fox Searchlight Pictures
6816         Siempre Viva Productions
6818              Borderline Presents
6819        Les Productions du Trésor
Name: company, Length: 2182, dtype: object

In [10]:
df4 = df[df['company']=='Columbia Pictures Corporation']['budget'].replace(0.0,a)

In [11]:
df4

0         8000000.0
35              0.0
59              0.0
82       12000000.0
101             0.0
           ...     
4522     55000000.0
4529            0.0
4556     20000000.0
4661    110000000.0
5255     10000000.0
Name: budget, Length: 166, dtype: float64

This imputation did not work as the value of 0.0 was not replaced by the mean. Instead of imputing the budget with an average value, I have preferred to ignore the rows with missing budgets. This leads to a loss of 1/3 of the data, but plugging in the average value may affect the roi(return on investement). As roi is dependent on budget values. 

df3 dataframe already has the values with the missing budget values removed

Return on investement can be considered as the ratio of gross to budget.

In [12]:
df3['roi']= df['gross']/df['budget']

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df3.sort_values(by = 'roi',ascending = False, inplace = True)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df3

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,roi
4683,15000.0,Solana Films,USA,Oren Peli,Horror,107918810.0,Paranormal Activity,R,2009-10-16,86,6.3,Katie Featherston,195668,Oren Peli,2007,7194.587333
2885,60000.0,Haxan Films,USA,Daniel Myrick,Horror,140539099.0,The Blair Witch Project,R,1999-07-30,81,6.4,Heather Donahue,202691,Daniel Myrick,1999,2342.318317
2123,25000.0,Brothers McMullen Productions,USA,Edward Burns,Comedy,10246600.0,The Brothers McMullen,R,1995-08-09,98,6.6,Jack Mulcahy,6598,Edward Burns,1995,409.864000
1388,7000.0,Columbia Pictures Corporation,USA,Robert Rodriguez,Action,2040920.0,El Mariachi,R,1993-02-26,81,6.9,Carlos Gallardo,55226,Robert Rodriguez,1992,291.560000
3806,120000.0,Plunge Pictures LLC,USA,Chris Kentis,Adventure,30610863.0,Open Water,R,2004-08-20,79,5.7,Blanchard Ryan,44989,Chris Kentis,2003,255.090525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,10000000.0,Middle Fork Productions,USA,Richard Harrah,Thriller,1785.0,The Canyon,R,2009-10-23,100,5.9,Eion Bailey,3453,Steve Allrich,2009,0.000179
5869,12000000.0,H2O Motion Pictures,Canada,David Weaver,Crime,1744.0,The Samaritan,R,2012-04-20,90,5.6,Samuel L. Jackson,8339,Elan Mastai,2012,0.000145
5664,94000000.0,Beijing New Picture Film Co.,China,Yimou Zhang,Drama,9213.0,The Flowers of War,R,2011-12-16,146,7.6,Christian Bale,41876,Heng Liu,2011,0.000098
5076,47000000.0,Pan Européenne,Belgium,Jaco Van Dormael,Drama,3600.0,Mr. Nobody,R,2013-09-26,141,7.9,Jared Leto,173190,Jaco Van Dormael,2009,0.000077


In [15]:
gnr_grp = df3.groupby(['genre'])

In [16]:
gnr_grp['roi','votes'].agg('mean').sort_values(by = 'votes')

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,roi,votes
genre,,
Musical,0.004476,1008.000000
Family,0.488875,8387.833333
Thriller,0.649603,10241.625000
Western,0.971773,18227.000000
Fantasy,1.608544,43848.517241
Romance,1.693938,46274.800000
Comedy,2.592252,60397.266412
Horror,45.742273,63209.385965
Drama,2.274116,84158.136192


Horror films have the highest return on investment, the number of movies made in horror genre is also to be considred as sometimes the sample may be too small to give the correct inference.

In [17]:
df['genre'].value_counts()

Comedy       2080
Drama        1444
Action       1331
Crime         522
Adventure     392
Biography     359
Horror        277
Animation     277
Mystery        38
Fantasy        32
Thriller       18
Romance        15
Family         14
Sci-Fi         13
Musical         4
War             2
Western         2
Name: genre, dtype: int64

The number of horror movies is also satisfactory. To check whether outlier values affect the mean we can compute median to confirm our interpretation.

In [18]:
gnr_grp['roi','votes'].agg('median').sort_values(by = 'votes')

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,roi,votes
genre,,
Musical,0.004476,1008.0
Thriller,0.242314,6231.5
Romance,1.244933,6451.0
Family,0.641666,6494.5
Western,0.971773,18227.0
Fantasy,1.142639,21333.0
Comedy,1.133715,31278.5
Drama,0.724004,34250.0
Horror,1.781617,39842.0


Median value of horror movies is also high, hence horror movies give the most return on investement. Also the genre mystery has highest number of votes. 

In [19]:
filt = (df3['genre']=='Horror')#considers the genre that gives most returns
df4 = df3[filt]

In [20]:
filt = (df4['roi'] >1.5)#considers only those movies that were profitable
df4 = df4[filt]

In [21]:
print(df4['director'].agg('mode'))
print(df4['star'].agg('mode'))
print(df4['writer'].agg('mode'))

0    James Wan
dtype: object
0    Donald Pleasence
1          Tobin Bell
dtype: object
0    Leigh Whannell
dtype: object


Hence the recepie for producing a high grossing film is to produce a film of genre Horror and select the above professionals in it.

For popularity of a movie we can consider the votes it received and rank the movies accordingly.

In [22]:
filt = (df3['genre'] == 'Mystery') #considers the genre that gives most votes
df5 = df3[filt]
filt = (df5['roi'] >1.5)  #considers only those movies that were profitable
df5 = df5[filt]

In [23]:
import numpy as np

In [24]:
a = (np.array(df5['director'].agg('mode')))
a

array(['Alan J. Pakula', 'Alan Rudolph', 'Christopher Nolan',
       'Duncan Jones', 'J.J. Abrams', 'James Mangold', 'Martin Scorsese',
       'Neil Burger', 'Olatunde Osunsanmi', 'Rob Cohen', 'Ron Howard',
       'Terry Gilliam', 'Wes Craven'], dtype=object)

In [25]:
b = np.array(df5['star'].agg('mode'))
c = np.array(df5['writer'].agg('mode'))

Here we have a tie between many stars, directors and writers. It might happen that a particular star, director and writer trio work well than a random choice. So while selecting, people out of the trio are to be avoided.

In [26]:
df6 = df5[(df5['director'].isin(a))]
df6 = df6[(df6['writer'].isin(c))]
df6 = df6[(df6['star'].isin(b))]
df6.sort_values(by = 'votes')
df6.iloc[0][['genre','director','writer','star']]

genre              Mystery
director         Rob Cohen
writer       Barbara Curry
star        Jennifer Lopez
Name: 6506, dtype: object

This gives the trio of director, writer and star that works well for the genre Mystery. This should make the movie popular.

Suppose some of the star/director or writer is not available for the movie, we can select the next trio or select the next respective professional from our dataframe as it has ranked them according to votes.

The code below gives the next best trio and the third best star for the mystery genre. 

In [27]:
print(df6.iloc[1][['genre','director','writer','star']])
print(df6.iloc[2]['star'])

genre              Mystery
director    Alan J. Pakula
writer         Scott Turow
star         Harrison Ford
Name: 918, dtype: object
Bradley Cooper


Similarly we can select the best professionals in horror genre for a high roi movie. We have high roi movies in dataframe df4. We'll print the best professionals and the second best star in the code below.

In [28]:
print(df4.iloc[0][['genre','director','writer','star']])
print(df4.iloc[1]['star'])

genre                  Horror
director            Oren Peli
writer              Oren Peli
star        Katie Featherston
Name: 4683, dtype: object
Heather Donahue


A successful producer may be defined as someone who gets high roi and his movies are popular. Consider the movies which are popular and have high return on investement. Dataframe df3 has nonmissing values.

In [29]:
filt = (df3['roi'] >1.5)#considers only those movies that were profitable
df7 = df3[filt].sort_values(by='votes',ascending=False)

We'll now consider the country in which these movies were produced. 

In [30]:
df7['country'].mode()

0    USA
dtype: object

Most films were made in USA, hence the producer should stay in USA for making high grossing and popular movies.

Limitations of the results :

1.The model has not accounted inflation. Since the movies are from the year 1986 till 2016.

2.The model is based on averages, medians and modes taken for different groups. No statistical learning technique has been used.

3.Almost 1/3 of the data has been dropped.

4.The professionls required for creating desirable movies may not be actively working right now.

5.Nonlinear relation in different variables has not been accounted for.

6.Categorical values can be encoded and used in models to give better results.